<a href="https://colab.research.google.com/github/Akashgola123/Ai_Document_catagry/blob/main/Doc_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit PyPDF2 easyocr pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using

In [3]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 64.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00


In [4]:


!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 1s (216 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
!mkdir text_image_dir


In [6]:
def pdf_to_image(pdf_path, output_dir):
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)
  pages = convert_from_path(pdf_path)
  for i, page in enumerate(pages):
      output_path = os.path.join(output_dir, f'output_{i}.png')
      print(output_path)
      page.save(output_path, 'PNG')
  return [os.path.join(output_dir, f'output_{i}.png') for i in range(len(pages))]


In [7]:
%%writefile app.py
import streamlit as st
from PyPDF2 import PdfReader
import easyocr
from pdf2image import convert_from_path
import os
import tempfile
import transformers
import torch

def pdf_to_image(pdf_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    pages = convert_from_path(pdf_path)
    for i, page in enumerate(pages):
        output_path = os.path.join(output_dir, f'output_{i}.png')
        print(output_path)
        page.save(output_path, 'PNG')
    return [os.path.join(output_dir, f'output_{i}.png') for i in range(len(pages))]
def extract_pdf(pdf_path):
    pdf =  PdfReader(pdf_path)
    text = ""
    images = []

    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text

    if not text:
        print("No text found in PDF. The PDF will be converted into images for OCR.")

    return text, images
def ocr_image(image_path):
    reader = easyocr.Reader(['en'])
    image_text = reader.readtext(image_path)
    ocr_text = ""
    for detection in image_text:
        text = detection[1]
        ocr_text += text + " "
    return ocr_text.strip()
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")
def clean_text(text):
    doc = nlp(text)
    cleaned_text = []
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            cleaned_text.append(token.lemma_)
    return " ".join(cleaned_text)
def process_document(text, task, query=None,target_language=None):
    models = {
        'extract_information': {
            'prompt': f"""
            Extract the following information from the text:
            1. Entities (names, dates, locations, organizations)
            2. Relationships between entities
            3. Summarize key information

            Text:
            {text}

            Extracted Information:
            """
        },
        'classify_document': {
            'prompt': f"""
            Classify the following document into one of the predefined categories: invoice, receipt, report, letter, other.

            Document:
            {text}

            Category:
            """
        },
        'translate_text': {
            'prompt': f"""
            Translate the following text into {target_language}.

            Text:
            {text}

            Translation:
            """
        },
        'ask_question': {
            'prompt': f"""
            Answer the following question based on the text:

            Question:
            {query}

            Text:
            {text}

            Answer:
            """
        },
        'specific_extraction': {
            'prompt': f"""
            Extract the following specific information from the text:

            Request:
            {query}

            Text:
            {text}

            Extracted Information:
            """
        },
        'review_extraction': {
            'prompt': f"""
            Review the following extracted data based on the text and provide feedback:

            Extracted Data:
            {query}

            Text:
            {text}

            Feedback:
            """
        }
    }

    if task not in models:
        return "Error: Invalid task."

    if task == 'translate_text' and not target_language:
        return "Error: Target language is required for translation task."

    if task in ['ask_question', 'specific_extraction', 'review_extraction'] and not query:
        return "Error: Query is required for the specified task."

    model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={
            "torch_dtype": torch.float16,
            "quantization_config": {"load_in_4bit": True},
            "low_cpu_mem_usage": True,
        },
    )

    prompt = models[task]['prompt']
    messages = [
        {"role": "system", "content": "You are a helpful assistant!"},
        {"role": "user", "content": prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        # pipeline.tokenizer.convert_tokens_to_ids("")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]


# Main function that run All functions in the correct order

# # Part 3:  User Interaction via a Chatbot Interface (OpenAI Chat API and Streamlit)

def main():
    file = st.file_uploader("Upload a PDF file", type="pdf")
    lang = st.selectbox("Select the target language for translation", ["French", "German", "Spanish", "Italian", "Portuguese", "Dutch", "Russian", "Chinese", "Japanese", "Korean","English"])
    Query = st.text_input("Enter your Question")
    Query1 = st.text_input("Enter your specific_extraction")
    Query2 = st.text_input("Enter your review_extraction")
    if file is not None:
    # Save the uploaded file to a temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
        temp_file.write(file.read())
        temp_file.close()

        # Define the output directory for images
        text_image_dir = '/content/text_image_dir/'  # Specify your output directory here

        # Convert PDF to images
        image_files = pdf_to_image(temp_file.name, text_image_dir)
        ocr_result = ""
        for image_file in image_files:
            ocr_result += ocr_image(image_file) + "\n"

        clean = clean_text(ocr_result)
        task = 'extract_information'
        result = process_document(clean, task)
        task = 'classify_document'
        result = process_document(result, task)
        task = 'translate_text'
        target_language = lang
        result = process_document(result, task, target_language=target_language)

        # Write the list of image files to the Streamlit app
        st.write(result)
        # query,specific_extraction,review_extraction
        task = 'ask_question'
        query = Query
        result = process_document(clean, task, query=query)
        st.write(result)
        task = 'specific_extraction'
        query = Query1
        result = process_document(clean, task, query=query)
        st.write(result)
        task = 'review_extraction'
        query = Query2
        result = process_document(clean, task, query=query)

        st.write(result)
        # Cleanup temporary file
        os.remove(temp_file.name)

if __name__ == "__main__":
    main()


Writing app.py


In [8]:
!wget -q -O - ipv4.icanhazip.com

34.105.113.219


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.113.219:8501

npx: installed 22 in 3.464s
your url is: https://old-impalas-tie.loca.lt
/content/text_image_dir/output_0.png
Progress: |██████████████████████████████████████------------| 77.1% Complete/content/text_image_dir/output_0.png
Progress: |████████████████████████████████████████----------| 80.3% CompleteDownloading detection model, please wait. This may take several minutes depending upon your network connection.
Progress: |███████-------------------------------------------| 15.7% Complete2024-06-25 07:02:33.946 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 589, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 232, in <module>
    main()
  File "/content/app.py", line 201, i